Hypothesis: We believe there's a correlation between temperature and the incidence of crime in Silver Spring, Maryland


In [ ]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime

In [ ]:
# Save file path to variable
crimeMD_path = "Crime_MD.csv"
# silverspring_path = "silver_spring_climate.csv"

# Read with Pandas
crime_df = pd.read_csv(crimeMD_path, low_memory=False)
# silverspring_temp_df = pd.read_csv(silverspring_path)


In [ ]:
# Pulling out only the columns of interest
main_crime_df = crime_df[['Crime Name1', 'Crime Name2', 'Crime Name3', 'City', 'Start_Date_Time']]

In [ ]:
# Remove crimeless data and changing columns name 
main_crime_df = main_crime_df.loc[main_crime_df['Crime Name1'] != 'Not a Crime']
clean_crime_df = main_crime_df.rename(columns={'Crime Name1': 'Crime Main Category',
                                             'Crime Name2': 'Crime Sub Category',
                                             'Start_Date_Time': 'Date & Time of Crime'})

clean_crime_df['Date & Time of Crime']= pd.to_datetime(clean_crime_df['Date & Time of Crime'])
clean_crime_df['City'].value_counts()
silverspring_crime_df = clean_crime_df.loc[clean_crime_df['City'] == 'SILVER SPRING']
silverspring_crime_df

In [ ]:
# Figure out the timeline of interest
data_first_date = silverspring_crime_df['Date & Time of Crime'].min()
data_last_date = silverspring_crime_df['Date & Time of Crime'].max()

print(data_first_date)
print(data_last_date)

# Throw away 2016 and 2020 
silverspring_clean_df = silverspring_crime_df.loc[(silverspring_crime_df['Date & Time of Crime'] > '2016-12-31 23:59:59') & (silverspring_crime_df['Date & Time of Crime'] < '2020-01-01 00:00:00')]

In [ ]:
silverspring_clean_df

In [ ]:
# Extract year from string format date
silverspring_clean_df['Year'] = pd.DatetimeIndex(silverspring_clean_df['Date & Time of Crime']).year
silverspring_clean_df['Month'] = pd.DatetimeIndex(silverspring_clean_df['Date & Time of Crime']).month

silverspring_clean_df.head(3)

In [ ]:
# Dan's code starts here - Property/Other

In [ ]:
# Dan's code ends here

In [ ]:
# Cynthia's code starts here - Person

Sub-Hypothesis: We believe that as temperature decreases to a certain point during the winter, the crime against person will also decreases. This is because people are less likely to come out during the winter time when it gets too cold, making crime occur less.


In [ ]:
# Running this doesn't work and mess up the whole dataframe....
# clean_crime_df = clean_crime_df['Year'].apply(str)
# clean_crime_df = clean_crime_df['Month'].apply(str)

In [ ]:
# Create Crime Against Person df
person_crime_df = clean_crime_df.loc[(clean_crime_df['Crime Main Category'] == 'Crime Against Person'), :]
person_crime_df['City'].value_counts()

In [ ]:
# Create different df for different year (2017-2019)
person_crime_2017df = person_crime_df.loc[person_crime_df['Year'] == 2017]
person_crime_2018df = person_crime_df.loc[person_crime_df['Year'] == 2018]
person_crime_2019df = person_crime_df.loc[person_crime_df['Year'] == 2019]

In [ ]:
# Using Groupby
crimemonthgroup = person_crime_df.groupby('Month')
personcrime_count_month = crimemonthgroup['Month'].count()
personcrime_count_month
df = personcrime_count_month
# personcrime_count_month.plot(kind='scatter', title='Numbers of Crimes Against Person for each Month from 2017-2020')

## To do scatter, need to get the count of each month, the # of crime
### x = month
### y = count
##### DO this for all the years

In [ ]:
# Question: Do I need to sort the DF before plotting it to get the month in order?

# Number of crime in each month for all three years
crimect = person_crime_df['Month'].value_counts()
crimectplot = crimect.plot(kind='bar', title='Numbers of Crimes Against Person for each Month from 2017-2020')
plt.xlabel('Month')
plt.ylabel('Number of Crimes')

In [ ]:
# Number of crime in each month for 2017
crimect_2017 = person_crime_2017df['Month'].value_counts()
crimect2017plot = crimect_2017.plot(kind='bar', title='Numbers of Crimes Against Person for each Month in 2017')
plt.xlabel('Month')
plt.ylabel('Number of Crimes')

In [ ]:
# Plotting incidence of crime VS months
# def personcrime(df, x):
    # Bar plot

In [ ]:
# Number of crimes in each city
# person_crime_city = person_crime_df.groupby('City')['City'].count()
# person_crime_city

# person_crime_city.plot(kind='bar', title='Numbers of Crime Against Person in each City', figsize=(8,8))
# plt.xlabel('City')
# plt.ylabel("Number of Cases")

# plt.xlim(-0.75, len(person_crime_city))
# plt.ylim(0, max(person_crime_city)+100)

# plt.tight_layout()
# plt.show()

# From here, maybe do a location

In [ ]:
# # Look at sub category of crime -- ain't going there -_-
# person_crime_stat = person_crime_df.groupby('Crime Sub Category')['Crime Sub Category'].count()
# person_crime_stat

In [ ]:
# person_crime_stat.plot(kind='bar', title='Count of Different Types of Crime Against Person')
# plt.xlabel('Types of Crime Against Person')
# plt.ylabel("Number of Cases")

# plt.xlim(-0.75, len(person_crime_stat))
# plt.ylim(0, max(person_crime_stat)+100)

# plt.show()

In [ ]:
# Cynthia's code ends here

In [ ]:
# Rose's code starts here - Society

In [ ]:
# Rose's code ends here

In [ ]:
# Paul's code starts here - API!!!

In [ ]:
# Paul's code ends here